In [ ]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import metrics

In [ ]:
#Dataset
data = pd.read_csv("/kaggle/input/ccdata/CC GENERAL.csv")
print(data.shape)
print(data.head())

In [ ]:
#Preprocessing
print(data.describe())
plt.figure(figsize=(10,10))
sns.boxplot(data=data)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Dropping CUST_ID
data.drop(['CUST_ID'], axis=1, inplace=True)
print(data.shape)


In [ ]:
#MissingValue
print(data.isnull().sum())
data['MINIMUM_PAYMENTS'].fillna(data['MINIMUM_PAYMENTS'].mean(),inplace=True)
data['CREDIT_LIMIT'].fillna(data['CREDIT_LIMIT'].mean(),inplace=True)
print(data.isnull().sum())

In [ ]:
#Outlier according to z-score
z = np.abs(stats.zscore(data))
print(z)
print(np.where(z > 3))
data_1 = data[(z < 3).all(axis=1)]
print(data.shape)
print(data_1.shape)
print(data_1.head())

In [ ]:
#Normalizing
scale = StandardScaler()
X = scale.fit_transform(data_1)
names = data_1.columns
data_2 = pd.DataFrame(X, columns=names)
print(data_2.head())

In [ ]:
#clustering

In [ ]:
#elbow
Sum_of_squared_distances=[]
k= range(1,16)
for i in k:
    kmean= KMeans(i)
    kmean.fit(data_2)
    Sum_of_squared_distances.append(kmean.inertia_)
plt.plot(k, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
#K-means
kmeans = KMeans(n_clusters=6)
kmeans.fit(data_2)
labels=kmeans.labels_
print(kmeans.cluster_centers_)
clusters=pd.concat([data_2, pd.DataFrame({'cluster':labels})], axis=1)
print(clusters.shape)
print(clusters.head())
print(clusters['cluster'])

In [ ]:
#Visualization
sns.countplot(x='cluster', data=clusters)

for c in data_1:
    grid= sns.FacetGrid(clusters, col='cluster')
    grid= grid.map(plt.hist, c)
plt.show()


In [ ]:
#Classification
print(clusters.shape)
print(clusters['cluster'].unique())
print(clusters.groupby('cluster').size())

In [ ]:
#Split Dataset
X = clusters [['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES','ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE','PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
    'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY','CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
    'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']]
y= clusters[['cluster']]
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3)
print(X_train)
print(X_test)


In [ ]:
#Decision_Tree
model= DecisionTreeClassifier(criterion="entropy")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
#Confusion_Matrix
print(metrics.confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
